<a href="https://colab.research.google.com/github/depinsetel/DEPIN-DATA/blob/main/NOTEBOOK%20FINAL%20km%20de%20UBS%20para%20PoP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Importação

In [ ]:
# Instalação de geobr com dados do Brasil e etc
!pip install geobr geopandas shapely folium
!pip install unidecode




# 2- Importação

In [ ]:
# Importando as bibliotecas necessárias
import geopandas as gpd
from shapely.geometry import Point
import geobr
import folium
import pandas as pd
import unidecode

#3 Função que checa se o munícpio está dentro ou fora do limite territorial municipal do geobr

In [9]:
# Função para verificar se um ponto está dentro dos limites de um município
def verificar_ponto_em_municipio(row):
    LAT_a = row['NU_LATITUDE']
    LON_a = row['NU_LONGITUDE']

    # Normalizar o nome do município removendo a acentuação e convertendo para minúsculas
    mun_alvo = unidecode.unidecode(row['Município']).lower()
    UF_alvo = row['UF']

    # Carregar os limites do município de interesse usando a biblioteca geobr
    municipios = geobr.read_municipality(code_muni='all', year=2020)

    # Aplicar a normalização de texto para o nome do município
    municipios['name_muni'] = municipios['name_muni'].apply(lambda x: unidecode.unidecode(x).lower())

    # Filtrar o município de interesse pelo nome e unidade da federação
    municipio_alvo = municipios[(municipios['name_muni'] == mun_alvo) & (municipios['abbrev_state'] == UF_alvo)]

    # Verificar se o município alvo foi encontrado
    if municipio_alvo.empty:
        print(f'Município "{mun_alvo}" não encontrado em "{UF_alvo}"')
        return False

    # Extrair o polígono do município alvo
    poligono_municipio = municipio_alvo.geometry.iloc[0]

    # Criar um objeto de ponto a partir das coordenadas fornecidas
    ponto = Point(LON_a, LAT_a)

    # Verificar se o ponto está dentro do polígono do município
    if poligono_municipio.contains(ponto):
        print(f'O ponto ({LAT_a}, {LON_a}) ESTÁ DENTRO do município de "{mun_alvo}" em "{UF_alvo}"')
        return True
    else:
        print(f'O ponto ({LAT_a}, {LON_a}) está F____O____R____A do município _____ "{mun_alvo}" em "{UF_alvo}"')
        return False

# 4 Entrada de dados de cerca de 22 mil UBS desconectadas ( TENHA DISPONÍVEL TODOS OS ARQUIVOS CARREGADOS)

In [7]:
# Ler o arquivo em um DataFrame do pandas utilizando a codificação
dfALFA = pd.read_excel("22k desconetadas PLANILHA.xlsx")
display(dfALFA)

,CNES,Nome Fantasia,Razão Social,Telefone,Usuário,Atualização,Email,UF,Município,Endereço Completo,Status Unidade,Conexão Internet,NU_LATITUDE,NU_LONGITUDE
0,2947110,ACADEMIA DE SAUDE MARIA DA LUZ,ACADEMIA DE SAUDE,NaN,SCNES,20/06/2022,semsa.dados@gmail.com,AC,TARAUACA,"S/N ANTONIO PINTO,TARAUACA,AC",Habilitada,Não Conectada,-8.161000,-70.766000
1,2947129,ACADEMIA DE SAUDE MARILIA VIEIRA,ACADEMIA DE SAUDE,NaN,SCNES,20/06/2022,semsa.dados@gmail.com,AC,TARAUACA,"S/N CORCOVADO,TARAUACA,AC",Habilitada,Não Conectada,-8.171952,-70.683177
2,2938162,ACADEMIA MUNICIPAL DE SAUDE,ACADEMIA DE SAUDE DE EPITACIOLANDIA,NaN,SCNES,08/09/2022,NaN,AC,EPITACIOLANDIA,"S/N RUA SAO SEBASTIAO,EPITACIOLANDIA,AC",Habilitada,Não Conectada,-11.029000,-68.741000
3,2947102,ACADEMIA DE SAUDE JOSE LEONCIO,ACADEMIA DE SAUDE TARAUACA,NaN,SCNES,20/06/2022,semsa.dados@gmail.com,AC,TARAUACA,"S/N AVENIDA AVELINO LEAL,TARAUACA,AC",Habilitada,Não Conectada,-8.150980,-70.781543
4,7619278,AEROBRAN TAXI AEREO,AEROBRAN TAXI AEREO LTDA,68 33222393,SCNES,06/09/2023,NaN,AC,CRUZEIRO DO SUL,"35 DESEMBARGADOR TAVORA,CRUZEIRO DO SUL,AC",Habilitada,Não Conectada,-7.636207,-72.667144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,2370360,UNIDADE BASICA DE SAUDE MUCIO BARBOSA TOCANTIN...,PREFEITURA MUNICIPAL DE TOCANTINOPOLIS,(63)34712203,FCESS,23/02/2023,tocantinopolis@saude.to.gov.br,TO,TOCANTINOPOLIS,"S/N RUA BELA VISTA,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.318562,-47.419975
21996,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,PREFEITURA MUNICIPAL DE XAMBIOA,(63)92927981,FCESS,01/02/2024,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N POVOADO MANCHAO MEIO,XAMBIOA,TO",Habilitada,Não Conectada,-6.413843,-48.528349
21997,7200447,UNIDADE BASICA DE SAUDE VEREADOR SALMEIRON ROCHA,PREFEITURA MUNICIPAL DE XAMBIOA,(63)99491875,FCESS,01/02/2024,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N RUA 03,XAMBIOA,TO",Habilitada,Não Conectada,-6.416423,-48.537898
21998,7125348,UNIDADE BASICA DE SAUDE DR WALTER HALLEY M AMADO,UNIDADE BASICA DE SAUDE DR WALTER HALLEY M AMADO,63 3474-1138,FCESTO,27/09/2023,araguatins@saude.to.gov.br,TO,ARAGUATINS,"S/N RUA 13 DE OUTUBRO,ARAGUATINS,TO",Habilitada,Não Conectada,-5.654081,-48.107243


In [ ]:
''' Eencerramos com aa colocção de uma coluna de STATUS DE PERTENCIMENTO das coordenadas de 2
    2 mil UBS ao território do município ou não & Aplicamos a função de cehcagem da territorialidade
    para cada linha/UBS do DataFrame e salvamos o resultado em uma nova coluna Ponto_Esta_No_Municipio'

'''
dfALFA['Ponto_Esta_No_Municipio'] = dfALFA.apply(verificar_ponto_em_municipio, axis=1)



In [11]:
dfALFA.shape[0]

22000

In [12]:
'''
Salvamos pro precaução
'''

dfALFA.to_excel("22k UBS Desconetadas PLANILHA Com TRUE_FALSE.xlsx", index=False)



---



# 5 A seguir, iremos carregar um arquivo com 12377 UBS que vieram do Minstério da Saúde com coordenadas erradas.Ele será df2


In [13]:
# Ler o arquivo Excel em um DataFrame do pandas __ 12377 UBS que vieram do Minstério da Saúde com coordenadas erradas
df2 = pd.read_excel("UBS_Conectividade_PAC_final__cod_mun.xlsx")
display(df2)

,UF,Município,Codigo Município,CNES,Nome Fantasia,Natureza Jurídica,Tipo Estabelecimento,Tipo Unidade,Telefone,Email,...,Gesac,MEC - Qual a velocidade de conexão da internet (Mbps) instalada n,MEC - Tipo de conexão,MEC - Velocidade é ideal?,Medidor - Velocidade média de download (Mbps),Medidor - Velocidade média de upload (Mbps),MegaEdu - Presença de Infra Fixa,Escolas.Prestadora PBLE-CER,Anatel Escolas.Situação PBLE-CER,Anatel Escolas.Velocidade desejada (Mbps)
0,AC,ACRELANDIA,1200013.0,3382745,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,Sim,Até 2Mbps,Satélite,Não,'-,'-,2. Acesso inadequado - menos de 20 km,'-,'-,225.0
1,AC,ACRELANDIA,1200013.0,3393984,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,Sim,Até 2Mbps,Satélite,Não,'-,'-,2. Acesso inadequado - menos de 20 km,'-,'-,225.0
2,AC,ACRELANDIA,1200013.0,3638685,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832321426,NaN,...,Sim,Até 2Mbps,Satélite,Não,'-,'-,2. Acesso inadequado - menos de 20 km,'-,'-,225.0
3,AC,ACRELANDIA,1200013.0,5403669,UNIDADE DE SAUDE DA FAMILIA REDENCAO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,Sim,Até 2Mbps,Satélite,Não,'-,'-,2. Acesso inadequado - menos de 20 km,'-,'-,225.0
4,AC,ASSIS BRASIL,1200054.0,2003066,UNIDADE BASICA DE SAUDE TEREZINHA BATISTA DOS ...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6835481414,semsab.ab@gmail.com,...,'-,De 10 Mbps a 20 Mbps,ADSL,Não,'-,'-,1. Acesso adequado - fibra,OI,Instalada,547.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12371,TO,TOCANTINOPOLIS,1721208.0,2370352,UNIDADE BASICA DE SAUDE ALTO BONITO TOCANTINOP...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717020,tocantinopolis@saude.to.gov.br,...,'-,De 20 Mbps a 50 Mbps,3G/4G,Não,'-,'-,1. Acesso adequado - fibra,OI,Instalada,136.0
12372,TO,TOCANTINOPOLIS,1721208.0,3904083,UNIDADE BASICA DE SAUDE ANA VINA TOCANTINOPOLIS,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717022,tocantinopolis@saude.to.gov.br,...,'-,De 50 Mbps a 100 Mbps,Fibra ótica,Sim,59.2,30,1. Acesso adequado - fibra,OI,Instalada,251.0
12373,TO,XAMBIOA,1722107.0,7170149,UNIDADE BASICA DE SAUDE ANASTACIO VIEIRA DOS S...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,'-,De 50 Mbps a 100 Mbps,Satélite,Sim,'-,'-,1. Acesso adequado - fibra,OI,Instalada,102.0
12374,TO,XAMBIOA,1722107.0,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,'-,De 50 Mbps a 100 Mbps,Satélite,Sim,'-,'-,1. Acesso adequado - fibra,OI,Instalada,102.0


In [25]:
'''
# Agora vamos gerar df3, por merge, acrescentando nas colunas de df2,
todas colunas  de dfALFA usando CNES como chave comum em ambos os dataframes.
Depois eliminamos de df3 as colunas Latitude e Longitude
'''
# Merge os dois DataFrames usando 'CNES' como chave comum
df3 = pd.merge(df2, dfALFA, on='CNES', how='left')

# Eliminar as colunas 'Latitude' e 'Longitude' de df3
df3 = df3.drop(['Latitude', 'Longitude'], axis=1)

# Exibir as primeiras linhas de df3
display(df3.tail())


,UF_x,Município_x,Codigo Município,CNES,Nome Fantasia_x,Natureza Jurídica,Tipo Estabelecimento,Tipo Unidade,Telefone_x,Email_x,...,Atualização,Email_y,UF_y,Município_y,Endereço Completo,Status Unidade,Conexão Internet,NU_LATITUDE,NU_LONGITUDE,Ponto_Esta_No_Municipio
12371,TO,TOCANTINOPOLIS,1721208.0,2370352,UNIDADE BASICA DE SAUDE ALTO BONITO TOCANTINOP...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717020,tocantinopolis@saude.to.gov.br,...,25/05/2023,tocantinopolis@saude.to.gov.br,TO,TOCANTINOPOLIS,"1927 RUA DARCI GOMES MARINHO,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.346180,-47.424760,True
12372,TO,TOCANTINOPOLIS,1721208.0,3904083,UNIDADE BASICA DE SAUDE ANA VINA TOCANTINOPOLIS,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717022,tocantinopolis@saude.to.gov.br,...,23/02/2023,tocantinopolis@saude.to.gov.br,TO,TOCANTINOPOLIS,"2079 RUA CHILE,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.320055,-47.434072,True
12373,TO,XAMBIOA,1722107.0,7170149,UNIDADE BASICA DE SAUDE ANASTACIO VIEIRA DOS S...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,01/02/2024,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N POVOADO CHAPADA,XAMBIOA,TO",Habilitada,Não Conectada,-6.414035,-48.529251,True
12374,TO,XAMBIOA,1722107.0,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,01/02/2024,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N POVOADO MANCHAO MEIO,XAMBIOA,TO",Habilitada,Não Conectada,-6.413843,-48.528349,True
12375,TO,XAMBIOA,1722107.0,7200447,UNIDADE BASICA DE SAUDE VEREADOR SALMEIRON ROCHA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6399491875,xambioa@saude.to.gov.br,...,01/02/2024,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N RUA 03,XAMBIOA,TO",Habilitada,Não Conectada,-6.416423,-48.537898,True


In [15]:
# Salvar o DataFrame como um arquivo Excel
df3.to_excel("TO_RAFAEL_12371_UBS_COORDENDAS_OK_COM_TRUE_FALSE.xlsx", index=False)

#6 Agora vamos abrir Base Backhauls.xlsx com o nome UBSPoP. Esse dataframe conterá coordenadas de um suposto ponto de presença  único "coordenadas"

In [16]:
# Ler o arquivo Excel em um DataFrame do pandas
UBSPoP  = pd.read_excel("Base Backhauls.xlsx")
display(UBSPoP)

,id,cod_municipio,operadora,tipo,origem_dado,coordenadas,chave_operadora,grupo_economico,nome_municipio,uf,tipo_original
0,33434,1100023,Vivo,Fibra,planilha_2021,POINT (-62.957255222993716 -9.951890006604886),VIVO,TELEFONICA,Ariquemes,RO,Fibra
1,33435,1100023,Abrint Abramult,Fibra,planilha_2021,POINT (-62.957255222993716 -9.951890006604886),ABRINT ABRAMULT,ABRINT ABRAMULT,Ariquemes,RO,Fibra
2,33436,1100023,Solintel,Fibra,planilha_2021,POINT (-62.957255222993716 -9.951890006604886),SOLINTEL,SOLINTEL,Ariquemes,RO,Fibra
3,33433,1100023,TIM,Fibra,planilha_2021,POINT (-62.957255222993716 -9.951890006604886),TIM,TELECOM ITALIA,Ariquemes,RO,Fibra
4,33437,1100031,Claro,Rádio,planilha_2021,POINT (-60.63980722013742 -13.474878553376326),CLARO,TELECOM AMERICAS,Cabixi,RO,Rádio
...,...,...,...,...,...,...,...,...,...,...,...
29617,32948,4305116,BERTOLINI & GUZZO LTDA,Fibra,planilha_2021,POINT (-52.00308852867675 -27.775438173833336),BERTOLINI GUZZO,BERTOLINI GUZZO,Centenário,RS,Fibra
29618,31058,2921401,CCNET- SERVIÇOS DE CONSULTORIA EM TECNOLOGIA D...,Fibra,planilha_2021,POINT (-40.714508801340095 -10.776984968481397),CCNET SERVICOS DE CONSULTORIA EM TECNOLOGIA DA...,CCNET SERVICOS DE CONSULTORIA EM TECNOLOGIA DA...,Mirangaba,BA,Fibra
29619,31087,2924058,TUX NET - MATERIAIS E SERVIÇOS DE INFORMÁTICA ...,Fibra,planilha_2021,POINT (-39.609677312753526 -11.87203536924399),TUX NET MATERIAIS E SERVICOS DE INFORMATICA,TUX NET MATERIAIS E SERVICOS DE INFORMATICA,Pé de Serra,BA,Fibra
29620,31125,2928950,TUX NET - MATERIAIS E SERVIÇOS DE INFORMÁTICA ...,Fibra,planilha_2021,POINT (-39.58038849478569 -11.454355144200816),TUX NET MATERIAIS E SERVICOS DE INFORMATICA,TUX NET MATERIAIS E SERVICOS DE INFORMATICA,São Domingos,BA,Fibra


## PROCESSAMENTO



---



# BLOCO 1 - MERGE de df2 e dfALFA

In [91]:
# Merge os dois DataFrames usando 'CNES' como chave comum
df3 = pd.merge(df2, dfALFA, on='CNES', how='left')

# Eliminar as colunas 'Latitude' e 'Longitude' de df3
df3 = df3.drop(['Latitude', 'Longitude'], axis=1)
df3.rename(columns={'Codigo Município': 'cod_municipio'}, inplace=True)

# Realiza o merge utilizando como chave os campos "cod_municipio" de df3 e dfMunPoP
df4 = pd.merge(df3, dfMunPoP, on='cod_municipio', how='left')

# Verifica se a quantidade de linhas de df4 é igual à quantidade de linhas de df3
assert df4.shape[0] == df3.shape[0], "A quantidade de linhas de df4 não é igual à quantidade de linhas de df3"

# Visualiza o DataFrame df4
display(df4)


,UF_x,Município_x,cod_municipio,CNES,Nome Fantasia_x,Natureza Jurídica,Tipo Estabelecimento,Tipo Unidade,Telefone_x,Email_x,...,Email_y,UF_y,Município_y,Endereço Completo,Status Unidade,Conexão Internet,NU_LATITUDE,NU_LONGITUDE,Ponto_Esta_No_Municipio,coordenadas
0,AC,ACRELANDIA,1200013.0,3382745,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,NaN,AC,ACRELANDIA,"16 BR 364 KM 90 RAMAL GRANADA KM 16,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837)
1,AC,ACRELANDIA,1200013.0,3393984,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,NaN,AC,ACRELANDIA,"284 RUA 07 QUEDAS,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837)
2,AC,ACRELANDIA,1200013.0,3638685,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832321426,NaN,...,NaN,AC,ACRELANDIA,"114 BR 364 KM 114,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837)
3,AC,ACRELANDIA,1200013.0,5403669,UNIDADE DE SAUDE DA FAMILIA REDENCAO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,NaN,AC,ACRELANDIA,"AC 475 VILA REDENCAOI,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837)
4,AC,ASSIS BRASIL,1200054.0,2003066,UNIDADE BASICA DE SAUDE TEREZINHA BATISTA DOS ...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6835481414,semsab.ab@gmail.com,...,semsab.ab@gmail.com,AC,ASSIS BRASIL,"622 RUA FRANCISCA DAS CHAGAS,ASSIS BRASIL,AC",Habilitada,Não Conectada,-10.940669,-69.561204,True,POINT (-70.00568369859224 -10.7797588600449)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12371,TO,TOCANTINOPOLIS,1721208.0,2370352,UNIDADE BASICA DE SAUDE ALTO BONITO TOCANTINOP...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717020,tocantinopolis@saude.to.gov.br,...,tocantinopolis@saude.to.gov.br,TO,TOCANTINOPOLIS,"1927 RUA DARCI GOMES MARINHO,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.346180,-47.424760,True,POINT (-47.538071737457216 -6.256849812182799)
12372,TO,TOCANTINOPOLIS,1721208.0,3904083,UNIDADE BASICA DE SAUDE ANA VINA TOCANTINOPOLIS,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717022,tocantinopolis@saude.to.gov.br,...,tocantinopolis@saude.to.gov.br,TO,TOCANTINOPOLIS,"2079 RUA CHILE,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.320055,-47.434072,True,POINT (-47.538071737457216 -6.256849812182799)
12373,TO,XAMBIOA,1722107.0,7170149,UNIDADE BASICA DE SAUDE ANASTACIO VIEIRA DOS S...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N POVOADO CHAPADA,XAMBIOA,TO",Habilitada,Não Conectada,-6.414035,-48.529251,True,POINT (-48.451035122407546 -6.545436604304118)
12374,TO,XAMBIOA,1722107.0,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,XAMBIOA@SAUDE.TO.GOV.BR,TO,XAMBIOA,"S/N POVOADO MANCHAO MEIO,XAMBIOA,TO",Habilitada,Não Conectada,-6.413843,-48.528349,True,POINT (-48.451035122407546 -6.545436604304118)


# BLOCO 2 - Trata as coordendas dos PoPs Pontos de Presença para tranformar em uma TUPLA

In [95]:
from shapely.wkt import loads
import pandas as pd

def transform_coord(coord):
    if isinstance(coord, str):  # Verifica se o valor é uma string
        coord = coord.replace('POINT (', '').replace(')', '')  # Remove o texto 'POINT' e os parênteses
        x, y = map(float, coord.split())  # Divide a string em duas partes e converte para floats
        return x, y  # Retorna a tupla de floats
    else:
        return None  # Retorna None se o valor não for uma string

df4['NU_COORD'] = df4.apply(lambda row: (row['NU_LONGITUDE'], row['NU_LATITUDE']), axis=1)

# Aplica a função à coluna 'coordenadas' e cria a coluna 'PoP_Tupla' com os valores transformados
df4['PoP_Tupla'] = df4['coordenadas'].apply(transform_coord)

# Exibindo o DataFrame df4 com os novos campos
display(df4)


,UF_x,Município_x,cod_municipio,CNES,Nome Fantasia_x,Natureza Jurídica,Tipo Estabelecimento,Tipo Unidade,Telefone_x,Email_x,...,Município_y,Endereço Completo,Status Unidade,Conexão Internet,NU_LATITUDE,NU_LONGITUDE,Ponto_Esta_No_Municipio,coordenadas,NU_COORD,PoP_Tupla
0,AC,ACRELANDIA,1200013.0,3382745,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,ACRELANDIA,"16 BR 364 KM 90 RAMAL GRANADA KM 16,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)"
1,AC,ACRELANDIA,1200013.0,3393984,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,ACRELANDIA,"284 RUA 07 QUEDAS,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)"
2,AC,ACRELANDIA,1200013.0,3638685,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832321426,NaN,...,ACRELANDIA,"114 BR 364 KM 114,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)"
3,AC,ACRELANDIA,1200013.0,5403669,UNIDADE DE SAUDE DA FAMILIA REDENCAO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,ACRELANDIA,"AC 475 VILA REDENCAOI,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)"
4,AC,ASSIS BRASIL,1200054.0,2003066,UNIDADE BASICA DE SAUDE TEREZINHA BATISTA DOS ...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6835481414,semsab.ab@gmail.com,...,ASSIS BRASIL,"622 RUA FRANCISCA DAS CHAGAS,ASSIS BRASIL,AC",Habilitada,Não Conectada,-10.940669,-69.561204,True,POINT (-70.00568369859224 -10.7797588600449),"(-69.561204, -10.940669)","(-70.00568369859224, -10.7797588600449)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12371,TO,TOCANTINOPOLIS,1721208.0,2370352,UNIDADE BASICA DE SAUDE ALTO BONITO TOCANTINOP...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717020,tocantinopolis@saude.to.gov.br,...,TOCANTINOPOLIS,"1927 RUA DARCI GOMES MARINHO,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.346180,-47.424760,True,POINT (-47.538071737457216 -6.256849812182799),"(-47.42475986480713, -6.346179776126681)","(-47.538071737457216, -6.256849812182799)"
12372,TO,TOCANTINOPOLIS,1721208.0,3904083,UNIDADE BASICA DE SAUDE ANA VINA TOCANTINOPOLIS,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717022,tocantinopolis@saude.to.gov.br,...,TOCANTINOPOLIS,"2079 RUA CHILE,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.320055,-47.434072,True,POINT (-47.538071737457216 -6.256849812182799),"(-47.43407249450684, -6.32005453979295)","(-47.538071737457216, -6.256849812182799)"
12373,TO,XAMBIOA,1722107.0,7170149,UNIDADE BASICA DE SAUDE ANASTACIO VIEIRA DOS S...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,XAMBIOA,"S/N POVOADO CHAPADA,XAMBIOA,TO",Habilitada,Não Conectada,-6.414035,-48.529251,True,POINT (-48.451035122407546 -6.545436604304118),"(-48.52925062179565, -6.414035207121636)","(-48.451035122407546, -6.545436604304118)"
12374,TO,XAMBIOA,1722107.0,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,XAMBIOA,"S/N POVOADO MANCHAO MEIO,XAMBIOA,TO",Habilitada,Não Conectada,-6.413843,-48.528349,True,POINT (-48.451035122407546 -6.545436604304118),"(-48.52834939956665, -6.413843296848917)","(-48.451035122407546, -6.545436604304118)"


# BLOCO 3 - Calcula a distância de cada UBS para o Pop do município corespondente

In [104]:
from geopy.distance import geodesic

# Função para calcular a distância geodésica entre dois pontos
def calcular_distancia(row):
    # Extrai as coordenadas dos pontos NU_COORD e PoP_Tupla
    coord_nu = row['NU_COORD']
    coord_pop = row['PoP_Tupla']

    try:
        # Calcula a distância geodésica entre os dois pontos
        distancia = geodesic(coord_nu, coord_pop).kilometers
    except ValueError as e:
        # Se houver um erro de valor (latitude fora do intervalo [-90, 90]), retorna NaN
        distancia = float('nan')
        distancia = round(distancia, 3)

    return distancia

# Aplica a função para calcular a distância geodésica e cria a coluna DIST_UBS_TIL_PoP
df4['DIST_UBS_TIL_PoP'] = df4.apply(calcular_distancia, axis=1)

# Exibe o DataFrame df4 com a nova coluna DIST_UBS_TIL_PoP
display(df4)


/usr/local/lib/python3.10/dist-packages/geopy/point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


,UF_x,Município_x,cod_municipio,CNES,Nome Fantasia_x,Natureza Jurídica,Tipo Estabelecimento,Tipo Unidade,Telefone_x,Email_x,...,Endereço Completo,Status Unidade,Conexão Internet,NU_LATITUDE,NU_LONGITUDE,Ponto_Esta_No_Municipio,coordenadas,NU_COORD,PoP_Tupla,DIST_UBS_TIL_PoP
0,AC,ACRELANDIA,1200013.0,3382745,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,"16 BR 364 KM 90 RAMAL GRANADA KM 16,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)",8.809525
1,AC,ACRELANDIA,1200013.0,3393984,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,"284 RUA 07 QUEDAS,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)",8.809525
2,AC,ACRELANDIA,1200013.0,3638685,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832321426,NaN,...,"114 BR 364 KM 114,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)",8.809525
3,AC,ACRELANDIA,1200013.0,5403669,UNIDADE DE SAUDE DA FAMILIA REDENCAO,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6832351426,NaN,...,"AC 475 VILA REDENCAOI,ACRELANDIA,AC",Habilitada,Não Conectada,-9.828000,-66.883000,True,POINT (-66.95016769820441 -9.933934857755837),"(-66.883, -9.828)","(-66.95016769820441, -9.933934857755837)",8.809525
4,AC,ASSIS BRASIL,1200054.0,2003066,UNIDADE BASICA DE SAUDE TEREZINHA BATISTA DOS ...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6835481414,semsab.ab@gmail.com,...,"622 RUA FRANCISCA DAS CHAGAS,ASSIS BRASIL,AC",Habilitada,Não Conectada,-10.940669,-69.561204,True,POINT (-70.00568369859224 -10.7797588600449),"(-69.561204, -10.940669)","(-70.00568369859224, -10.7797588600449)",49.972926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12371,TO,TOCANTINOPOLIS,1721208.0,2370352,UNIDADE BASICA DE SAUDE ALTO BONITO TOCANTINOP...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717020,tocantinopolis@saude.to.gov.br,...,"1927 RUA DARCI GOMES MARINHO,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.346180,-47.424760,True,POINT (-47.538071737457216 -6.256849812182799),"(-47.42475986480713, -6.346179776126681)","(-47.538071737457216, -6.256849812182799)",14.284303
12372,TO,TOCANTINOPOLIS,1721208.0,3904083,UNIDADE BASICA DE SAUDE ANA VINA TOCANTINOPOLIS,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6334717022,tocantinopolis@saude.to.gov.br,...,"2079 RUA CHILE,TOCANTINOPOLIS,TO",Habilitada,Não Conectada,-6.320055,-47.434072,True,POINT (-47.538071737457216 -6.256849812182799),"(-47.43407249450684, -6.32005453979295)","(-47.538071737457216, -6.256849812182799)",12.505382
12373,TO,XAMBIOA,1722107.0,7170149,UNIDADE BASICA DE SAUDE ANASTACIO VIEIRA DOS S...,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,"S/N POVOADO CHAPADA,XAMBIOA,TO",Habilitada,Não Conectada,-6.414035,-48.529251,True,POINT (-48.451035122407546 -6.545436604304118),"(-48.52925062179565, -6.414035207121636)","(-48.451035122407546, -6.545436604304118)",13.037734
12374,TO,XAMBIOA,1722107.0,7170157,UNIDADE BASICA DE SAUDE ESPEDITO ROQUE DE SANTANA,MUNICIPIO,UNIDADE BASICA DE SAUDE,CENTRO DE SAUDE/UNIDADE BASICA,6392927981,xambioa@saude.to.gov.br,...,"S/N POVOADO MANCHAO MEIO,XAMBIOA,TO",Habilitada,Não Conectada,-6.413843,-48.528349,True,POINT (-48.451035122407546 -6.545436604304118),"(-48.52834939956665, -6.413843296848917)","(-48.451035122407546, -6.545436604304118)",12.981784


# BLOCO FINAL - SALVA O ARQUIVO FINAL COM TODAS AS INFORMAÇÔES

In [102]:
df4.to_excel("TO_RAFAEL_12376_km_UBS_ATE PoP.xlsx", index=False)